# Neighbours in Toronto

### TASK 1

In [47]:

from bs4 import BeautifulSoup


import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

ImportError: No module named 'folium'

In [27]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
bSoup = BeautifulSoup(data, 'html.parser')
#bSoup

In [10]:
postalCodeList = []
boroughList = []
neighborList = []

for row in bSoup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborList.append(cells[2].text.rstrip('\n'))

In [17]:
#The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
toronto_info = [('PostalCode',postalCodeList), ('Borough', boroughList),('Neighbourhood', neighborList)]
toronto_df = pd.DataFrame.from_dict(dict(toronto_info))
toronto_df.head(20)

,Borough,Neighbourhood,PostalCode
0,Not assigned,Not assigned,M1A
1,Not assigned,Not assigned,M2A
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,Harbourfront,M5A
5,Downtown Toronto,Regent Park,M5A
6,North York,Lawrence Heights,M6A
7,North York,Lawrence Manor,M6A
8,Queen's Park,Not assigned,M7A
9,Not assigned,Not assigned,M8A


In [19]:
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
toronto_df_filtered = toronto_df[toronto_df.Borough != 'Not assigned'].reset_index(drop=True)
toronto_df_filtered.head(20)

,Borough,Neighbourhood,PostalCode
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,Harbourfront,M5A
3,Downtown Toronto,Regent Park,M5A
4,North York,Lawrence Heights,M6A
5,North York,Lawrence Manor,M6A
6,Queen's Park,Not assigned,M7A
7,Etobicoke,Islington Avenue,M9A
8,Scarborough,Rouge,M1B
9,Scarborough,Malvern,M1B


In [38]:
#append comma and sort/group based on postal code and  Borough
toronto_df_joined = toronto_df_filtered.groupby(['PostalCode','Borough'],as_index=False).agg(lambda x: ','.join(x))
toronto_df_joined.head(12)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [26]:
# use the .shape method to print the number of rows of your dataframe.
toronto_df_joined.shape

(103, 3)

### TASK 2

In [32]:
#using the e Geocoder package
url="http://cocl.us/Geospatial_data"
coordinates = pd.read_csv(url)
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [37]:
toronto_df_join = toronto_df_joined.set_index('PostalCode')
coordinates_join = coordinates.set_index('Postal Code')
toronto_df_coordinates = pd.concat([toronto_df_join, coordinates_join], axis=1, join='inner')
toronto_df_coordinates.index.name = 'PostalCode'

toronto_df_coordinates.reset_index(inplace=True)
toronto_df_coordinates.head(12)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [36]:
toronto_df_coordinates.shape

(103, 5)

### TASK 3

In [40]:
#Explore and cluster the neighborhoods in Toronto. 
# Get the coordinates of Toronto City


address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="tl-toronto-neigh")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of Toronto City are {}, {}.'.format(latitude, longitude))


The coordinates of Toronto City are 43.653963, -79.387207.


In [44]:
#work with only boroughs that contain the word Toronto and then create clusters

toronto_boroughs_list = ['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']
toronto_word_df = toronto_df_coordinates[toronto_df_coordinates['Borough'].isin(toronto_boroughs_list)].reset_index(drop=True)

print(toronto_word_df.shape)

toronto_word_df.head(12)




(38, 5)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049


In [45]:
#Pass the coordinates to Foursquare
CLIENT_ID = 'NQ4QLPVACRENFRRHCTHAQY332WY4WRRXHWIFQAG1TUYUPM4Z' # your Foursquare ID
CLIENT_SECRET = 'W4UZSMLN1NT5HVNOCENZA3A1XS2PQCFGF4Q3VPFSRM1MDNFJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [54]:
#Explore
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough,neighborhood in zip(toronto_word_df['Latitude'], toronto_word_df['Longitude'], toronto_word_df['PostalCode'], toronto_word_df['Borough'],toronto_word_df['Neighbourhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [57]:
venues_df = pd.DataFrame(venues)
venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLat', 'BoroughLong', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(venues_df.shape)
venues_df.head(12)

(1700, 9)


,PostalCode,Borough,Neighborhood,BoroughLat,BoroughLong,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,St-Denis Studios Inc.,43.675031,-79.288022,Music Venue
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
5,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,Dolce Gelato,43.677773,-79.351187,Ice Cream Shop
6,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop
7,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant
8,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,Mezes,43.677962,-79.350196,Greek Restaurant
9,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,Messini Authentic Gyros,43.677827,-79.350569,Greek Restaurant


In [62]:
countBy_df = venues_df.groupby(['PostalCode', 'Borough', 'Neighborhood'])['VenueName'].count()
countBy_df

PostalCode  Borough           Neighborhood                                                                                        
M4E         East Toronto      The Beaches                                                                                               4
M4K         East Toronto      The Danforth West,Riverdale                                                                              44
M4L         East Toronto      The Beaches West,India Bazaar                                                                            22
M4M         East Toronto      Studio District                                                                                          37
M4N         Central Toronto   Lawrence Park                                                                                             4
M4P         Central Toronto   Davisville North                                                                                          7
M4R         Central Toronto   North Toron